<a href="https://colab.research.google.com/github/achrafBenHamou/Deep_Learning/blob/second_model/second_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers


     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
     |████████████████████████████████| 2.9MB 37.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7f4d85c2df72b05f530f2556bd9cfab9fe25c0ea5a57d0fc01eaf538e46595ac
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
from transformers import RobertaTokenizer
import tokenizers
import pickle
import re
import math
import numpy as np
import pandas as pd
from google.colab import files
print('TF version',tf.__version__)

TF version 2.4.0


In [5]:
uploaded = files.upload()
import io

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [14]:

train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
train.head()


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [64]:
## maximal lenth of a tweet
MAX_LEN = 96
'''
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file='../input/tf-roberta/vocab-roberta-base.json', 
    merges_file='../input/tf-roberta/merges-roberta-base.txt', 
    lowercase=True
)'''

from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

EPOCHS = 3 # originally 3
BATCH_SIZE = 32 # originally 32
PAD_ID = 1
LABEL_SMOOTHING = 0.1
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}


## remove html 
def remove_html(text):
    text = re.sub('https?://\S+|www\.\S+', '', text)
    return text
def scheduler(epoch):
    return 3e-5 * 0.2**epoch
reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
train

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [66]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')


In [20]:
train.shape[0]

27481

In [19]:
train.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [59]:
#train.drop([314]) 27480
train.loc[314] = train.loc[27480] 

In [60]:
train.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [61]:
# drop all rows with any NaN and NaT values
train2 = train.dropna()

In [26]:
text1 = " "+" ".join(train2.loc[314,'text'].split())
text1

' All this flirting going on - The ATG smiles. Yay. ((hugs))'

In [67]:
for k in range(train.shape[0]):    
    # FIND OVERLAP
    print("iteration ",k)
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
    print(enc)    
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc)+3] = [0, s_tok] + enc + [2]
    attention_mask[k,:len(enc)+3] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+2] = 1
        end_tokens[k,toks[-1]+2] = 1
    if k == 2:
        print(start_tokens[k])
        print(end_tokens[k])
        print(text1)
        print(input_ids[k])
        print(text2)
        a = np.argmax(start_tokens[k])
        b = np.argmax(end_tokens[k])
        man = tokenizer.encode(text1)
        print(tokenizer.decode(enc[a-2:b-1]))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
iteration  24981
[0, 14826, 6, 47, 58, 84, 158, 6, 151, 212, 31744, 328, 24, 12905, 29, 182, 628, 5267, 86, 53, 52, 4711, 62, 7, 972, 47, 2]
iteration  24982
[0, 2054, 640, 261, 13878, 4, 175, 860, 7, 213, 114, 47, 64, 4, 38, 1137, 961, 54, 1411, 7, 5817, 7, 1649, 24, 66, 4, 1648, 82, 54, 697, 89, 2]
iteration  24983
[0, 38, 12905, 119, 95, 7, 444, 31, 147, 47, 32, 6, 38, 23126, 213, 184, 734, 2]
iteration  24984
[0, 8976, 38, 95, 3903, 121, 3573, 12905, 29, 904, 94, 363, 6, 53, 38, 46910, 114, 38, 12905, 890, 202, 146, 24, 11, 142, 38, 12905, 548, 57, 3369, 7361, 160, 10, 319, 12056, 4, 2]
iteration  24985
[0, 5534, 54, 5090, 6, 127, 1099, 2]
iteration  24986
[0, 939, 657, 70, 2955, 13350, 20986, 324, 329, 13531, 2721, 2216, 8, 13350, 6, 8, 2793, 4, 33976, 905, 10, 31095, 486, 1717, 10, 31095, 740, 4987, 10717, 13531, 966, 172, 13516, 939, 1266, 24, 2]
iteration  24987
[0, 259, 741, 17137, 6, 42, 16, 45, 10,

In [68]:
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc)+3] = [0, s_tok] + enc + [2]
    attention_mask_t[k,:len(enc)+3] = 1


In [69]:
def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model
def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)
def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    batch_size = tf.shape(y_true)[0]
    LEN = ll 
    ind_row = tf.range(0, LEN)
    ones_matrix = tf.ones([batch_size, LEN])
    #print(K.int_shape(ones_matrix))
    ind_row = tf.cast(ind_row ,dtype = tf.float32)
    ones_matrix = tf.cast(ones_matrix ,dtype = tf.float32)     
    ind_matrix = ind_row * ones_matrix
    k1 = tf.cast(tf.math.argmax(y_true ,axis = 1) ,dtype = tf.float32)
    k2 = K.sum(y_pred * ind_matrix, axis=1)
    y_pred = tf.cast(y_pred ,dtype = tf.float32) 
    y_true = tf.cast(y_true ,dtype = tf.float32) 
    loss1 = tf.keras.losses.binary_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss2 = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean((loss1+loss2))
    return loss
def build_model():
    ##########config = RobertaConfig.from_pretrained('../input/tf-roberta/config-roberta-base.json')
    config = RobertaConfig.from_pretrained("roberta-base")
    ####bert_model = TFRobertaModel.from_pretrained('../input/tf-roberta/pretrained-roberta-base.h5',config=config)
    bert_model = TFRobertaModel.from_pretrained('roberta-base')
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(0.15)(x[0])
    x1 = tf.keras.layers.Conv1D(1536, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(128, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.15)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1536, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(128, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)

    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) 
    model.compile(loss=loss_fn ,optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model

In [70]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [73]:
import tensorflow.keras.backend as K
from transformers import RobertaConfig, RobertaModel, TFRobertaModel
import math

In [ ]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN) )
oof_end = np.zeros((input_ids.shape[0],MAX_LEN) )
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN) )
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN) )
skf = StratifiedKFold(n_splits=5,shuffle=True)#,random_state=SEED) #originally 5 splits
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    ##################K.clear_session()
    model, padded_model = build_model()
        
    #sv = tf.keras.callbacks.ModelCheckpoint(
    #    '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
    #    save_weights_only=True, mode='auto', save_freq='epoch')
    inpT = [input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]]
    targetT = [start_tokens[idxT,], end_tokens[idxT,]]
    inpV = [input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]]
    targetV = [start_tokens[idxV,], end_tokens[idxV,]]
    # sort the validation data
    shuffleV = np.int32(sorted(range(len(inpV[0])), key=lambda k: (inpV[0][k] == PAD_ID).sum(), reverse=True))
    inpV = [arr[shuffleV] for arr in inpV]
    targetV = [arr[shuffleV] for arr in targetV]
    weight_fn = '3-%s-roberta-%i.h5'%(VER,fold)
    for epoch in range(1, EPOCHS + 1):
        # sort and shuffle: We add random numbers to not have the same order in each epoch
        shuffleT = np.int32(sorted(range(len(inpT[0])), key=lambda k: (inpT[0][k] == PAD_ID).sum() + np.random.randint(-3, 3), reverse=True))
        # shuffle in batches, otherwise short batches will always come in the beginning of each epoch
        num_batches = math.ceil(len(shuffleT) / BATCH_SIZE)
        batch_inds = np.random.permutation(num_batches)
        shuffleT_ = []
        for batch_ind in batch_inds:
            shuffleT_.append(shuffleT[batch_ind * BATCH_SIZE: (batch_ind + 1) * BATCH_SIZE])
        shuffleT = np.concatenate(shuffleT_)
        # reorder the input data
        inpT = [arr[shuffleT] for arr in inpT]
        targetT = [arr[shuffleT] for arr in targetT]
        model.fit(inpT, targetT, 
            epochs=epoch, initial_epoch=epoch - 1, batch_size=BATCH_SIZE, verbose=DISPLAY,
            validation_data=(inpV, targetV), shuffle=False ,callbacks = [reduce_lr])  # don't shuffle in `fit`
        save_weights(model, weight_fn)

    print('Loading model...')
    # model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    load_weights(model, weight_fn)

    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = padded_model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            ##error
            ##st = tokenizer.decode(enc.ids[a-2:b-1])
            st = tokenizer.decode(enc[a-2:b-1])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    print()

#########################
### FOLD 1
#########################


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


 19/687 [..............................] - ETA: 1:53:35 - loss: 6.8822 - activation_20_loss: 3.4458 - activation_21_loss: 3.4364